# 1.필요한 라이브러리 설치

BERTopic : https://maartengr.github.io/BERTopic/index.html

In [1]:
!pip install bertopic
!pip install bertopic[visualization]
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 7.7 MB/s 
     |████████████████████████████████| 5.2 MB 67.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 662 kB 66.3 MB/s 
     |████████████████████████████████| 85 kB 6.7 MB/s 
     |████████████████████████████████| 88 kB 8.0 MB/s 
     |████████████████████████████████| 5.8 MB 48.4 MB/s 
     |████████████████████████████████| 1.3 MB 36.0 MB/s 
     |████████████████████████████████| 182 kB 70.9 MB/s 
     |████████████████████████████████| 7.6 MB 58.7 MB/s 
     |████████████████████████████████| 1.1 MB 58.6 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp38-cp38-linux_x86_64.whl size=2700845 sha256=51208511b1e85b8cc82fc15cc8b5951c48147fafcd1a8fd79d535546e465fea8
  Stored in directory: /root/.cache/

In [2]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 15.4 MB/s 
     |████████████████████████████████| 465 kB 68.7 MB/s 
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ bash -x
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb-release /etc/os-release
+ '[' '/etc/lsb-release:DISTRIB_ID=Ubuntu
/etc/lsb-release:DISTRIB_DESCRIPTION="Ubuntu 18.04.6 LTS"
/etc/os-release:NAME="Ubuntu"
/etc/os-release:ID=ubuntu
/etc/os-release:ID_LIKE=debian
/

# 2.데이터 전처리 및 업로드

In [3]:
# Kowiki 10개의 문단으로된 문서 다운로드
!wget https://raw.githubusercontent.com/byungjooyoo/Dataset/main/kowiki_10.txt
with open("kowiki_10.txt") as f:
  data = f.read()

--2022-12-17 21:52:04--  https://raw.githubusercontent.com/byungjooyoo/Dataset/main/kowiki_10.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55488 (54K) [text/plain]
Saving to: ‘kowiki_10.txt’

kowiki_10.txt       100%[===================>]  54.19K  --.-KB/s    in 0.003s  

2022-12-17 21:52:05 (15.2 MB/s) - ‘kowiki_10.txt’ saved [55488/55488]



In [4]:
# 각 문서를 sent_tokenize 함수를 이용하여 문장단위로 리스트화하여 편집
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
sents = sent_tokenize(data)
print(len(sents))
sents[-2:]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


407


['서양 음악에서는 도·레·미·파·솔·라·시로 된 7음계가 많이 쓰이지만 한국 전통 음악에는 황종(黃鍾)-미♭·태주(太蔟)-파·중려(仲呂)-라♭·임종(林鍾)-시♭·무역(無射)-레♭으로 된 5음계가 많이 쓰이고, 중국 전통 음악에는 궁-도·상-레·각-미·변치(變徵)-올림화(Fa )·치-솔·우-라·변궁(變宮)-시로 7음계를 많이 쓴다.',
 '한국 전통 음악에서는 5음계 외에도 3음계 또는 악계통에서는 7음계 등이 쓰인다.']

# 3.Mecab과 BERTopic을 이용한 토픽모형

참조모델 : https://huggingface.co/models

참조 : https://huggingface.co/sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens

Pretrained Models : https://www.sbert.net/docs/pretrained_models.html

In [8]:
from tqdm import tqdm
from konlpy.tag import Mecab
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
# 자체 토큰화 클래스 
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        # word_tokens = self.tagger.morphs(sent)
        word_tokens = self.tagger.nouns(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result
# Mecab을 이용한 토큰화와 모형 적합
custom_tokenizer = CustomTokenizer(Mecab())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens",
                 language='multiligual',
                 vectorizer_model=vectorizer,
                 nr_topics="auto")
topics, probs = model.fit_transform(sents)
model.get_topic_info()

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

,Topic,Count,Name
0,-1,32,-1_자극_세기_사이_보일
1,0,99,0_화학_물질_원자_화합물
2,1,75,1_방정식_맥스웰_법칙_자기
3,2,51,2_수학_구조_공간_분야
4,3,44,3_미국_대통령_카터_인권
5,4,38,4_집합_목록_상수_초월수
6,5,36,5_문학_감상_작품_이야기
7,6,16,6_체첸_공화국_러시아_러시아인
8,7,16,7_음계_음악_반음계_온음계


In [9]:
import pandas as pd
tt =[]
vect = CountVectorizer()
for sent in sents[:3]:
    tokenizer = custom_tokenizer(sent)
    tt.append(tokenizer)
    print(custom_tokenizer(sent))
    print(vect.fit_transform(tokenizer))


['카터', '제임스', '지미', '카터', '주니어', '민주당', '출신', '미국', '대통령']
  (0, 7)	1
  (1, 3)	1
  (2, 5)	1
  (3, 7)	1
  (4, 4)	1
  (5, 2)	1
  (6, 6)	1
  (7, 1)	1
  (8, 0)	1
['카터', '조지', '섬터', '카운티', '레인스', '마을']
  (0, 5)	1
  (1, 3)	1
  (2, 2)	1
  (3, 4)	1
  (4, 0)	1
  (5, 1)	1
['조지아', '공과', '대학교', '졸업']
  (0, 2)	1
  (1, 0)	1
  (2, 1)	1
  (3, 3)	1


In [10]:
# 토픽 분류에서 -1은 outlier를 모아놓은 것인데 이는 어느 토픽에도 포함시키기 모호한 문서들을 분류한 것이고 이는 통상 토픽으로 다루지 않음
# count는 주제별로 포함되는 문장의 개수
print(len(topics))
model.get_topic_info()

407


,Topic,Count,Name
0,-1,32,-1_자극_세기_사이_보일
1,0,99,0_화학_물질_원자_화합물
2,1,75,1_방정식_맥스웰_법칙_자기
3,2,51,2_수학_구조_공간_분야
4,3,44,3_미국_대통령_카터_인권
5,4,38,4_집합_목록_상수_초월수
6,5,36,5_문학_감상_작품_이야기
7,6,16,6_체첸_공화국_러시아_러시아인
8,7,16,7_음계_음악_반음계_온음계


In [11]:
model.visualize_topics()

In [12]:
model.visualize_heatmap()

In [13]:
for i in range(8):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('화학', 0.136016970485944), ('물질', 0.07586792030504802), ('원자', 0.07472408305904636), ('화합물', 0.07472408305904636), ('결합', 0.07369321380965312), ('반응', 0.06793535251122387), ('원소', 0.06312851388263524), ('유기', 0.05815826086169936), ('분석', 0.04911914769007884), ('금속', 0.04444996734660941)]
1 번째 토픽 : [('방정식', 0.17060279413790524), ('맥스웰', 0.16171038731820866), ('법칙', 0.08020158631653146), ('자기', 0.07552519076576747), ('헤비사이드', 0.05404149880701133), ('전류', 0.05404149880701133), ('논문', 0.051270680328554086), ('유도', 0.04803688782845452), ('전자기파', 0.04474449556201562), ('역선', 0.04474449556201562)]
2 번째 토픽 : [('수학', 0.1972952709749942), ('구조', 0.06694229432038255), ('공간', 0.06648045114363484), ('분야', 0.06586841983214838), ('발전', 0.05675914409290364), ('기하학', 0.055043518718456357), ('연구', 0.052790181987481886), ('개념', 0.047904305332471545), ('상수', 0.04414600096114728), ('대수학', 0.04252335828982237)]
3 번째 토픽 : [('미국', 0.11219207488844848), ('대통령', 0.09195561369886852), ('카터', 0.0772372

In [14]:
model.visualize_barchart()

In [15]:
# 토픽수를 4개로 고정 선택
model1 = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens",
                 language='multiligual',
                 vectorizer_model=vectorizer,
                 nr_topics=4)
topics, probs = model1.fit_transform(sents)
print(model1.get_topic_info())
model1.visualize_barchart()

   Topic  Count             Name
0     -1     91  -1_미국_체첸_대통령_카터
1      0    118   0_화학_물질_원자_화합물
2      1     76    1_문학_감상_작품_집합
3      2     71  2_방정식_맥스웰_자기_법칙
4      3     51    3_수학_구조_분야_공간


In [16]:
model1.visualize_heatmap()

# 4.모델 저장

In [24]:
# 모델 저장, 1.16G 용량
model1.save("topic_model")

# 5.주제 분류분석에 활용

In [25]:
#모델 업로드 및 사용
from bertopic import BERTopic
model2 = BERTopic.load("topic_model")
model2.visualize_barchart()

In [26]:
#새로운 분류가 필요한 문서
new_docs =['산문은 사건을 중심으로 그려진 이야기이며 인물의 성격을 묘사하는 것을 중심으로 한 소설이 있다.',
                   '이러한 물질을 물에 녹이면 이온은 물 분자에 의해 수화된다.']
model2.transform(new_docs)

([1, 0], array([0.57868911, 1.        ]))

In [30]:
# 주제별 문서
print(model2.get_topic_freq())

   Topic  Count
0      0    118
1     -1     91
2      1     76
3      2     71
4      3     51


In [31]:
# 주제별 대표적인 문장
model2.get_representative_docs()

{0: ['결합이 끊어질 때는 에너지가 흡수되고, 결합이 이어질 때는 에너지가 방출된다.', '생화학은 생물을 화학의 관점에서 다루는 학문이다.', '원자는 양성자와 중성자로 구성되어 있는 원자핵을 가지고 있으며 전자는 이 주변에 오비탈을 이루며 분포되어 있다.', '한국 전통 음악에서는 5음계 외에도 3음계 또는 악계통에서는 7음계 등이 쓰인다.', '반음계는 12개의 반음으로 이루어진 음계를 말한다.', '서양 음악에서는 도·레·미·파·솔·라·시로 된 7음계가 많이 쓰이지만 한국 전통 음악에는 황종(黃鍾)-미♭·태주(太蔟)-파·중려(仲呂)-라♭·임종(林鍾)-시♭·무역(無射)-레♭으로 된 5음계가 많이 쓰이고, 중국 전통 음악에는 궁-도·상-레·각-미·변치(變徵)-올림화(Fa )·치-솔·우-라·변궁(變宮)-시로 7음계를 많이 쓴다.'], 3: ['조선 후기의 한국의 수학은 실학자들을 중심으로 다시 발전하였고, 새로운 결과도 성취되었다.', '이런 발전은 그 뒤로도 계속되어, 16세기의 르네상스에 이르러서는 수학적 발전과 과학적 방법들의 상호 작용이 일어나, 혁명적인 연구들이 진행되며 인류 문명에 큰 영향을 미치게 되었다.', '수학을 이런 분야들에 적용한 응용수학은 그 결과로써 수학 자체의 발전을 이끌고 새로운 분야들을 낳았다.'], 1: ['이것은 고대 그리스 시대부터의 난제였던 원적문제가 불가능함을 보여주는 결과였다.', '예를 들어, formula_1는 무리수이지만 다음 이차방정식\n의 해이므로 초월수가 아니다.', '즉, 전하는 양전하이든 음전하이든 단독으로 존재할 수 있는 데 반해 자극은 홀극으로 존재할 수 없고, N극과 S극이 언제나 쌍으로 존재하여야 한다는 것이다.', '문학은 원래 문예(文藝)라고 부르는 것이 옳으며, 문학을 학문의 대상으로서 탐구하는 학문의 명칭 역시 문예학이다.', '산문은 사건을 중심으로 그려진 이야기, 근대 리얼리즘의 수법 이후 인물의 성격을 묘사하는 것을 중심으로 한 소설이 있다.', '더 나아가 보면, "문학"과 몇몇

In [29]:
#주제별 라벨
model2.generate_topic_labels()

['-1_미국_체첸_대통령', '0_화학_물질_원자', '1_문학_감상_작품', '2_방정식_맥스웰_자기', '3_수학_구조_분야']

In [23]:
model2.visualize_hierarchy()

# 6.다양한 Pretrained Model 적용

Pretrained Models : https://www.sbert.net/docs/pretrained_models.html

In [32]:
#문장단위 리스트를 명사만 추출한 리스트로 전처리
from konlpy.tag import Mecab
mecab = Mecab()
docs = [' '.join(mecab.nouns(sent)) for sent in sents]
docs[:2]

['카터 제임스 얼 지미 카터 주니어 년 월 일 민주당 출신 미국 번 대통령 년 년', '카터 조지 섬터 카운티 플 레인스 마을']

In [33]:
# 전처리한 문서를 사용하여 토픽 분석, 평균적으로 가장 좋은 Pretrained Model 적용
from tqdm import tqdm
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer("all-mpnet-base-v2")
model3 = BERTopic(embedding_model=sentence_model, language="multiligual", nr_topics= 4)
topics, probs = model3.fit_transform(docs)
print(model3.get_topic_info())
model3.visualize_barchart()

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

   Topic  Count             Name
0     -1    146  -1_화학_초월수_결합_원소
1      0     78    0_문학_국가_화학_원자
2      1     77    1_수학_상수_집합_화학
3      2     66  2_방정식_맥스웰_전류_법칙
4      3     40  3_감상_방정식_작용_맥스웰


In [34]:
#처리속도가 가장빠른 Pretrained Model 적용
sentence_model = SentenceTransformer("paraphrase-MiniLM-L3-v2")
model4 = BERTopic(embedding_model=sentence_model, language="multiligual", nr_topics= 4)
topics, probs = model4.fit_transform(docs)
print(model4.get_topic_info())
model4.visualize_barchart()

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

   Topic  Count               Name
0     -1     67  -1_맥스웰_화합물_방정식_유기
1      0    193      0_원소_반응_음계_화학
2      1     90      1_화학_물질_미국_연구
3      2     36     2_방정식_기준_국가_추측
4      3     21    3_방정식_맥스웰_전기_음악


In [37]:
new_doc = '추축 분석 방정식 물질'
# embeddings1 = sentence_model.encode([new_doc])
model4.transform(new_docs)

([1, -1], array([0.86629689, 0.        ]))

In [46]:
new_docs1 = [' '.join(mecab.nouns(sent)) for sent in new_docs]
print(new_docs1)
model4.transform(new_docs1)

['산문 사건 중심 이야기 인물 성격 묘사 것 중심 소설', '물질 물 이온 물 분자 수화']


([-1, 0], array([0.        , 0.81881703]))

In [42]:
model4.get_topic_info()

,Topic,Count,Name
0,-1,67,-1_맥스웰_화합물_방정식_유기
1,0,193,0_원소_반응_음계_화학
2,1,90,1_화학_물질_미국_연구
3,2,36,2_방정식_기준_국가_추측
4,3,21,3_방정식_맥스웰_전기_음악


In [43]:
model4.visualize_hierarchy()